# Homework 5

Make sure to fill in the `YOUR RESPONSE HERE`, and the posting in the Discussion at the end.

## In-class exercise

[Geocode](https://en.wikipedia.org/wiki/Address_geocoding) an address from Python using the [NYC Planning GeoSearch API](https://geosearch.planninglabs.nyc/docs/). Print out the latitude and longitude. Any NYC address is fine:

- Your own
- This building: 238 Thompson St, New York, NY 10012
- etc.

In [1]:
# your code here
import requests
geo = requests.get("https://geosearch.planninglabs.nyc/v1/autocomplete?text=61 troutman")
data = geo.json()
data["features"][0]["geometry"]["coordinates"]

[-73.931827, 40.698661]

In [2]:
import ipytest

ipytest.autoconfig()

## Coding

### Preface

This assignment intentionally has far less step-by-step guidance than previous homeworks. It's meant to be challenging. You have learned all the pieces necessary, now it's a matter of figuring out which you need and putting them together.

### The assignment

When doing mapping or spatial analysis, it's easier to work with geographic coordinates than addresses. The 311 dataset has the `latitude` and `longitude` filled in for many of the rows, but not all. [Enrich the dataset](https://www.redpointglobal.com/blog/what-is-data-enrichment/) by filling in those missing coordinates, where possible.

- Use the smaller dataset from homework 2: https://storage.googleapis.com/python-public-policy/data/cleaned_311_data_hw2.csv.zip
- Use the same API as above
- To avoid overwhelming the API, operate on a small subset of the data (no more than thirty addresses that need geocoding).

### Tips

- Start small: Try doing single requests to the API from code before trying to run it against a DataFrame.
- You're going to be doing API calls (`requests.get()`) row by row (`apply()`)
- You can use [this technique for updating multiple columns at once through an `apply()`](https://stackoverflow.com/a/51703667/358804).

In [3]:
# your code here
import pandas as pd
requests311 = pd.read_csv("https://storage.googleapis.com/python-public-policy/data/cleaned_311_data_hw2.csv.zip")

In [4]:
requests311.head()

,Unnamed: 0.1,Unnamed: 0,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)"
2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
filtered311 = requests311[requests311["latitude"].isna() & requests311["longitude"].isna() & requests311["incident_address"].notna() & requests311["borough"].notna()]
filtered311_30 = filtered311[['incident_address', 'borough', 'latitude', 'longitude']].head(30)
filtered311_30


,incident_address,borough,latitude,longitude
8,141 141 ST,QUEENS,NaN,NaN
36,HYLAN BOULEVARD,Unspecified,NaN,NaN
98,BALTIC STREET,BROOKLYN,NaN,NaN
121,101 101 ST,QUEENS,NaN,NaN
128,101 101 ST,QUEENS,NaN,NaN
129,101 101 ST,QUEENS,NaN,NaN
131,101 101 ST,QUEENS,NaN,NaN
206,AVENUE A,MANHATTAN,NaN,NaN
305,5TH 5TH AVE,BROOKLYN,NaN,NaN
408,65 STREET,BROOKLYN,NaN,NaN


In [84]:
#Create a new dataset that has the rows with missing long/lat values but has address and borough data, then use a def/return function to search the planninglabs.nyc page, then use that function in another def/return function to return coordinates. Then use .apply() to fill in the coordinates to the new dataset you made.
def coord(row):
    if isinstance(row, str):
        geo = requests.get("https://geosearch.planninglabs.nyc/v1/autocomplete?text=" + row)
        try:
            data = geo.json()
            latlong = data["features"][0]["geometry"]["coordinates"]
            print("latlong", latlong)
            return latlong[0], latlong[1]
        except:
            print("row:", row)
            return "invalid", "invalid"
    else:
        return "invalid", "invalid"


In [85]:
coord("101 101 st")

latlong [-73.8412, 40.684]


(-73.8412, 40.684)

In [86]:
filtered311_30.dtypes

incident_address     object
borough              object
latitude            float64
longitude           float64
latlong              object
dtype: object

In [96]:
#filtered311_30[['latitude', 'longitude']] = filtered311_30['incident_address'].apply(coord, axis=1, result_type='expand')
filtered311_30.apply(lambda x: filtered311_30['incident_address'], axis=0, result_type='expand')

,incident_address,borough,latitude,longitude,latlong
8,141 141 ST,141 141 ST,141 141 ST,141 141 ST,141 141 ST
36,HYLAN BOULEVARD,HYLAN BOULEVARD,HYLAN BOULEVARD,HYLAN BOULEVARD,HYLAN BOULEVARD
98,BALTIC STREET,BALTIC STREET,BALTIC STREET,BALTIC STREET,BALTIC STREET
121,101 101 ST,101 101 ST,101 101 ST,101 101 ST,101 101 ST
128,101 101 ST,101 101 ST,101 101 ST,101 101 ST,101 101 ST
129,101 101 ST,101 101 ST,101 101 ST,101 101 ST,101 101 ST
131,101 101 ST,101 101 ST,101 101 ST,101 101 ST,101 101 ST
206,AVENUE A,AVENUE A,AVENUE A,AVENUE A,AVENUE A
305,5TH 5TH AVE,5TH 5TH AVE,5TH 5TH AVE,5TH 5TH AVE,5TH 5TH AVE
408,65 STREET,65 STREET,65 STREET,65 STREET,65 STREET


In [93]:
filtered311_30

,incident_address,borough,latitude,longitude,latlong
8,141 141 ST,QUEENS,NaN,NaN,"[-73.940066, 40.818978]"
36,HYLAN BOULEVARD,Unspecified,NaN,NaN,"[-74.094801, 40.584488]"
98,BALTIC STREET,BROOKLYN,NaN,NaN,"[-73.992311, 40.685257]"
121,101 101 ST,QUEENS,NaN,NaN,"[-73.8412, 40.684]"
128,101 101 ST,QUEENS,NaN,NaN,"[-73.8412, 40.684]"
129,101 101 ST,QUEENS,NaN,NaN,"[-73.8412, 40.684]"
131,101 101 ST,QUEENS,NaN,NaN,"[-73.8412, 40.684]"
206,AVENUE A,MANHATTAN,NaN,NaN,"[-73.973074, 40.677368]"
305,5TH 5TH AVE,BROOKLYN,NaN,NaN,"[-73.99331, 40.661786]"
408,65 STREET,BROOKLYN,NaN,NaN,"[-74.01055, 40.631741]"


### Question

**What are the complicating factors** with this task? What could make it difficult to do comprehensively? Looking for a paragraph or some bullet points.

YOUR RESPONSE HERE

In [72]:
print(coord('HYLAN BOULEVARD'))

[-74.094801, 40.584488]


Now [turn in the assignment](https://python-public-policy.afeld.me/en/latest/README.html#turning-in-assignments).


## Participation

Reminder about the [between-class participation requirement](https://python-public-policy.afeld.me/en/latest/syllabus.html#participation).